In [ ]:
import os
import os.path
import random
from operator import add
from datetime import datetime, date, timedelta

import numpy as np
import pandas as pd
# Andrei: matplotlib and seabord are not used -> comment
# from matplotlib import pyplot as plt
# import seaborn as sns

import shutil

# Andrei: ema_workbench is not used -> so comment
# import ema_workbench
import time

# Andrei: scipy.integrate is not used -> so comment
# from scipy.integrate import simps
from numpy import trapz
from decimal import Decimal, ROUND_DOWN, ROUND_UP

import sqlite3

import plotly.express as px
import plotly.graph_objects as go

## Historical

In [ ]:

df_from_csv_dfq50_years_obs = pd.read_csv('dfq50_years_obs.csv')
arr_from_sql_dfq50_years_obs = df_from_csv_dfq50_years_obs['q50_years_obs'].values


df_from_csv_dfq75_years_obs = pd.read_csv('dfq75_years_obs.csv')
arr_from_sql_dfq75_years_obs = df_from_csv_dfq75_years_obs['q75_years_obs'].values

df_from_csv_dfq90_years_obs = pd.read_csv('dfq90_years_obs.csv')
arr_from_sql_dfq90_years_obs = df_from_csv_dfq90_years_obs['q90_years_obs'].values


In [ ]:

df_from_csv = pd.read_csv('array_data_matrix_HotDays_obs.csv')
arr_from_sql_matrix_HotDays_obs = df_from_csv.values.reshape((80, 41, 2))



In [ ]:
x_axis_obs = np.array([1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991,
       1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002,
       2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013,
       2014, 2015, 2016, 2017, 2018, 2019, 2020])

In [ ]:
values = ['1981',  '', '1983',  '',  '1985',  '', '1987', '', '1989', '', '1991', '', '1993', '', '1995', '', '1997', '', '1999',
 '',  '2001',  '',  '2003',  '', '2005', '', '2007', '', '2009', '', '2011', '', '2013', '', '2015', '', '2017', '', '2019',
 '']

In [ ]:
p_Step = 1
alpha_Fade_obs = 1
# Create figure object
fig = go.Figure()

for i in range(0, len(arr_from_sql_matrix_HotDays_obs), p_Step):
    a = arr_from_sql_matrix_HotDays_obs[i, 1:, 1:2].astype(float)
    if i == 0:
        fig.add_trace(go.Scatter(x=x_axis_obs, y=a[:,0], line=dict(color='green', dash='dash'), opacity=alpha_Fade_obs, name='Observed'))
    else:
        fig.add_trace(go.Scatter(x=x_axis_obs, y=a[:,0], line=dict(color='green', dash='dash'), opacity=alpha_Fade_obs, name='Sim {}'.format(i)))

fig.add_trace(go.Scatter(x=x_axis_obs, y=arr_from_sql_dfq50_years_obs.flatten(), line=dict(color='black', width=2.5), opacity=1, name='Median'))
fig.add_trace(go.Scatter(x=x_axis_obs, y=arr_from_sql_dfq75_years_obs.flatten(), line=dict(color='yellow', width=2.5), opacity=1, name='75th percentile'))
fig.add_trace(go.Scatter(x=x_axis_obs, y=arr_from_sql_dfq90_years_obs.flatten(), line=dict(color='red', width=2.5), opacity=1, name='90th percentile'))

# Customize x-axis
fig.update_xaxes(title='Historical number of Hot Days & Nights (M1) Over the Years', tickvals=x_axis_obs, ticktext=values, tickfont=dict(size=15), range=[1981, 2021], showgrid=True)

# Customize y-axis
fig.update_yaxes(title='Number of Hot Days & Nights', titlefont=dict(size=17), range=[0,30], showgrid=True)

# Add title to the figure
fig.update_layout(title='Zürich, Historical (1980-2020)', title_font=dict(size=20))

fig.show()


## RCP2.6

In [ ]:

df_from_csv_dfq50_years_26 = pd.read_csv('dfq50_years_26.csv')
arr_from_sql_dfq50_years_26 = df_from_csv_dfq50_years_26['q50_years_26'].values


df_from_csv_dfq75_years_26 = pd.read_csv('dfq75_years_26.csv')
arr_from_sql_dfq75_years_26 = df_from_csv_dfq75_years_26['q75_years_26'].values


df_from_csv_dfq90_years_26 = pd.read_csv('dfq90_years_26.csv')
arr_from_sql_dfq90_years_26 = df_from_csv_dfq90_years_26['q90_years_26'].values

In [ ]:

df_from_csv = pd.read_csv('array_data_matrix_HotDays_26.csv')
arr_from_sql_matrix_HotDays_26 = df_from_csv.values.reshape((5440, 120, 2))


In [ ]:
p_Step = 20
alpha_Fade = 1

In [ ]:
fig = go.Figure()

x_axis = np.arange(2020, 2101, step=1)

for i in range(0, len(arr_from_sql_matrix_HotDays_26), p_Step):
    a = arr_from_sql_matrix_HotDays_26[i, 1:, 1:2].astype(float)
    fig.add_trace(go.Scatter(x=x_axis, y=a[:,0], mode='lines', 
                             line=dict(color='green', dash='dash'), 
                             opacity=alpha_Fade, name=f'sim {i//p_Step+1}'))

fig.add_trace(go.Scatter(x=x_axis, y=arr_from_sql_dfq50_years_26[:], mode='lines', 
                         line=dict(color='black', width=1.5), 
                         opacity=1, name='median'))
fig.add_trace(go.Scatter(x=x_axis, y=arr_from_sql_dfq75_years_26[:], mode='lines', 
                         line=dict(color='blue', width=1.5), 
                         opacity=1, name='75th percentile'))
fig.add_trace(go.Scatter(x=x_axis, y=arr_from_sql_dfq90_years_26[:], mode='lines', 
                         line=dict(color='yellow', width=2), 
                         opacity=1, name='90th percentile'))

#Customize x-axis
fig.update_xaxes(tickvals=np.arange(2020, 2101, 10), tickfont=dict(size=20))

#Customize y-axis
fig.update_yaxes(title='Number of Hot Days & Nights (M1)', 
                 titlefont=dict(size=20), 
                 range=[0, 80], 
                 tickfont=dict(size=20))

#Set subplot title
fig.update_layout(title='Zürich, RCP2.6', 
                  title_font=dict(size=20),
                  xaxis_title='Number of Hot Days & Nights (M1) Over the Years in Zürich for RCP2.6  (2020-2100)',
                  yaxis_title='Number of Hot Days & Nights')

fig.show()

## RCP 4.5

In [ ]:

df_from_csv_dfq50_years_45 = pd.read_csv('dfq50_years_45.csv')
arr_from_sql_dfq50_years_45 = df_from_csv_dfq50_years_45['q50_years_45'].values


df_from_csv_dfq75_years_45 = pd.read_csv('dfq75_years_45.csv')
arr_from_sql_dfq75_years_45 = df_from_csv_dfq75_years_45['q75_years_45'].values


df_from_csv_dfq90_years_45 = pd.read_csv('dfq90_years_45.csv')
arr_from_sql_dfq90_years_45 = df_from_csv_dfq90_years_45['q90_years_45'].values



In [ ]:

df_from_csv = pd.read_csv('array_data_matrix_HotDays_45.csv')
arr_from_sql_matrix_HotDays_45 = df_from_csv.values.reshape((5440, 120, 2))



In [ ]:
fig = go.Figure()

x_axis = np.arange(2020, 2101, step=1)

for i in range(0, len(arr_from_sql_matrix_HotDays_45), p_Step):
    a = arr_from_sql_matrix_HotDays_45[i, 1:, 1:2].astype(float)
    fig.add_trace(go.Scatter(x=x_axis, y=a[:,0], mode='lines', 
                             line=dict(color='orange', dash='dash'), 
                             opacity=alpha_Fade, name=f'sim {i//p_Step+1}'))

fig.add_trace(go.Scatter(x=x_axis, y=arr_from_sql_dfq50_years_45[:], mode='lines', 
                         line=dict(color='black', width=1.5), 
                         opacity=1, name='Median'))
fig.add_trace(go.Scatter(x=x_axis, y=arr_from_sql_dfq75_years_45[:], mode='lines', 
                         line=dict(color='blue', width=1.5), 
                         opacity=1, name='75th percentile'))
fig.add_trace(go.Scatter(x=x_axis, y=arr_from_sql_dfq90_years_45[:], mode='lines', 
                         line=dict(color='yellow', width=2), 
                         opacity=1, name='90th percentile'))

#Customize x-axis
fig.update_xaxes(tickvals=np.arange(2020, 2101, 10), tickfont=dict(size=20))

#Customize y-axis
fig.update_yaxes(title='Number of Hot Days & Nights (M1)', 
                 titlefont=dict(size=20), 
                 range=[0, 80], 
                 tickfont=dict(size=20))

#Set subplot title
fig.update_layout(title='Zürich, RCP4.5', 
                  title_font=dict(size=20),
                  xaxis_title='Number of Hot Days & Nights (M1) Over the Years in Zürich for RCP4.5  (2020-2100)',
                  yaxis_title='Number of Hot Days & Nights')

fig.show()


## RCP 8.5

In [ ]:

df_from_csv_dfq50_years_85 = pd.read_csv('dfq50_years_85.csv')
arr_from_sql_dfq50_years_85 = df_from_csv_dfq50_years_85['q50_years_85'].values


df_from_csv_dfq75_years_85 = pd.read_csv('dfq75_years_85.csv')
arr_from_sql_dfq75_years_85 = df_from_csv_dfq75_years_85['q75_years_85'].values


df_from_csv_dfq90_years_85 = pd.read_csv('dfq90_years_85.csv')
arr_from_sql_dfq90_years_85 = df_from_csv_dfq90_years_85['q90_years_85'].values

#conn.close()

In [ ]:

# Convert the DataFrame back to a NumPy array
df_from_csv = pd.read_csv('array_data_matrix_HotDays_85.csv')
arr_from_sql_matrix_HotDays_85 = df_from_csv.values.reshape((5440, 120, 2))

# Close the database connection
#conn.close()

In [ ]:
fig = go.Figure()

x_axis = np.arange(2020, 2101, step=1)

for i in range(0, len(arr_from_sql_matrix_HotDays_85), p_Step):
    a = arr_from_sql_matrix_HotDays_85[i, 1:, 1:2].astype(float)
    fig.add_trace(go.Scatter(x=x_axis, y=a[:,0], mode='lines', 
                             line=dict(color='red', dash='dash'), 
                             opacity=alpha_Fade, name=f'sim {i//p_Step+1}'))

fig.add_trace(go.Scatter(x=x_axis, y=arr_from_sql_dfq50_years_85[:], mode='lines', 
                         line=dict(color='black', width=1.5), 
                         opacity=1, name='Median'))
fig.add_trace(go.Scatter(x=x_axis, y=arr_from_sql_dfq75_years_85[:], mode='lines', 
                         line=dict(color='blue', width=1.5), 
                         opacity=1, name='75th percentile'))
fig.add_trace(go.Scatter(x=x_axis, y=arr_from_sql_dfq90_years_85[:], mode='lines', 
                         line=dict(color='yellow', width=2), 
                         opacity=1, name='90th percentile'))

#Customize x-axis
fig.update_xaxes(tickvals=np.arange(2020, 2101, 10), tickfont=dict(size=20))

#Customize y-axis
fig.update_yaxes(title='Number of Hot Days & Nights (M1)', 
                 titlefont=dict(size=20), 
                 range=[0, 80], 
                 tickfont=dict(size=20))

#Set subplot title
fig.update_layout(title='Zürich, RCP8.5', 
                  title_font=dict(size=20),
                  xaxis_title='Number of Hot Days & Nights (M1) Over the Years in Zürich for RCP8.5  (2020-2100)',
                  yaxis_title='Number of Hot Days & Nights')

fig.show()

## END

# Box plots

### RCP 2.6

In [ ]:
import plotly.express as px
import pandas as pd

# Load data from CSV file
df = pd.read_csv("data_total_decadal_26.csv")
df2 = df.iloc[:, -8:]

# Set colors for the box plots
colors = ['green', 'green', 'green', 'green','green', 'green','green', 'green']

# Plot box plots with colors
fig = px.box(df2, color_discrete_sequence=colors)

# Customize layout
fig.update_layout(
    title='Zurich RCP2.6',
    yaxis_title='Annual number of Hot Days & Nights (M1)',
    boxmode='group'
)

# Show plot
fig.show()

### RCP 4.5

In [ ]:
import plotly.express as px
import pandas as pd

# Load data from CSV file
df = pd.read_csv("data_total_decadal_45.csv")
df2 = df.iloc[:, -8:]

# Set colors for the box plots
colors = ['orange', 'orange', 'orange', 'orange','orange', 'orange','orange', 'orange']

# Plot box plots with colors
fig = px.box(df2, color_discrete_sequence=colors)

# Customize layout
fig.update_layout(
    title='Zurich RCP4.5',
    yaxis_title='Annual number of Hot Days & Nights (M1)',
    boxmode='group'
)

# Show plot
fig.show()

### RCP 8.5

In [ ]:
import plotly.express as px
import pandas as pd

# Load data from CSV file
df = pd.read_csv("data_total_decadal_85.csv")
df2 = df.iloc[:, -8:]

# Set colors for the box plots
colors = ['red', 'red', 'red', 'red','red', 'red','red', 'red']

# Plot box plots with colors
fig = px.box(df2, color_discrete_sequence=colors)

# Customize layout
fig.update_layout(
    title='Zurich RCP8.5',
    yaxis_title='Annual number of Hot Days & Nights (M1)',
    boxmode='group'
)

# Show plot
fig.show()
